# Retrieve Injury Rates of MCTS Season Simulations 

(Files from compute cluster runs)

In [ ]:
import pandas as pd
import os
import numpy as np
import MDP_Team.MDP as MDP
from statistics import quantiles

In [ ]:
team_id = 58
team_directory = 'MCTS_Results/'+str(team_id)
files = os.listdir(team_directory)
team_squad = pd.read_csv('MDP_Team/Team_squads/'+str(team_id)+'.csv')
squad_player_ids = team_squad['player_name'].values
#team_rewards = pd.read_csv('MDP_Team/Team_rewards/'+str(team_id)+'.csv')

In [ ]:
positions = [[1,2,3,4],[5,6],[7,8],[9,10,11,12,13],[14,15],[16,17],[18,19]]
counts = [2,1,1,3,1,1,1]

def greedy_selection(selection_list, player_vaeps, num_selections, injury_dict):
    total_val = 0
    
    new_selection_list = []
    for i in range(len(selection_list)):
        if injury_dict[selection_list[i]] == False:
            new_selection_list.append(selection_list[i])
    
    while len(new_selection_list) < num_selections:
        new_selection_list.append(20)
            
    selection_values = player_vaeps[new_selection_list]
    max_indexes = np.argsort(selection_values)[-num_selections:]
    return [new_selection_list[i]-1 for i in sorted(max_indexes)]

In [ ]:
optimal_team_index=[]
dictionary = {key:False for key in range(1,20)}
squad_vaep = team_squad['vaep'].values
for p,c in zip(positions,counts):
    optimal_team_index += greedy_selection(p,squad_vaep,c,dictionary)

In [ ]:
starting_injuries_all = []
all_injuries_all = []
for f in files:
    try:
        team_df = pd.read_csv('MCTS_Results/'+str(team_id)+'/'+f)
        injury_changes = team_df.iloc[:,21:40].diff().fillna(0)
        injury_changes.columns = range(1,20)
        all_injuries = (injury_changes > 0).sum().sum()
        starting_index = optimal_team_index
        starting_injuries = injury_changes.iloc[:,starting_index]
        starting_inj_number = (starting_injuries>0).sum().sum()
        #starting_injuries_all.append(starting_inj_number)
        #all_injuries_all.append(all_injuries) 
        starting_injuries_all.append(starting_inj_number)
        all_injuries_all.append(all_injuries) 
    except:
        continue

In [ ]:
print("Total injuries: ", np.array(all_injuries_all).mean())
print("First Team injuries: ", np.array(starting_injuries_all).mean())

In [ ]:
print("Total injuries: ", np.percentile(np.array(all_injuries_all),25,method='linear'))
print("First Team injuries: ", np.percentile(np.array(starting_injuries_all),25,method='linear'))

In [ ]:
print("Total injuries: ", np.percentile(np.array(all_injuries_all),75,method='linear'))
print("First Team injuries: ", np.percentile(np.array(starting_injuries_all),75,method='linear'))